In [1]:
import numpy as np
from PIL import Image

In [2]:
def encode_label(label: int) -> list:
    code = [0] * 10
    code[label] = 1
    return code

In [3]:
for i in range(10):
    print(encode_label(i))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [4]:
import os
import tensorslow as slow

def load_data(path: str):
    labels = os.listdir(path)
    
    X, y = [], []
    for label in labels:
        label_encoded = encode_label(int(label))

        for sample in os.listdir(f"{path}/{label}")[:10]:
            png = Image.open(f"{path}/{label}/{sample}")
            arr = np.array(png)
            X.append(arr.flatten())
            y.append(label_encoded)

    return slow.tensor(np.array(X)), slow.tensor(np.array(y))

In [5]:
X_train, y_train = load_data("data/training")
X_test, y_test = load_data("data/testing")

In [6]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((100, 784), (100, 10), (100, 784), (100, 10))

In [7]:
batch_size = 32
input_size = 28 * 28
output_size = 10

h1 = input_size // 2
h2 = h1 // 4

In [8]:
def is_binary(arr):
    return np.all(np.logical_or(arr == 0, arr == 1))

In [9]:
def is_stochastic(arr):
    # assert len(arr[arr < 0]) == 0
    # assert len(arr[arr > 1]) == 0
    
    N, _ = arr.shape
    return np.allclose(np.sum(arr, axis=1), np.ones(N))

In [10]:
def softmax(x):
    

In [11]:
import tensorslow.nn as nn
from tensorslow.nn.activations import relu

class FeedForwardNet(nn.Module):
    def __init__(self):
        self.l1 = nn.layers.Dense(in_dim=input_size, out_dim=h1)
        self.l2 = nn.layers.Dense(in_dim=h1, out_dim=h2)
        self.l3 = nn.layers.Dense(in_dim=h2, out_dim=output_size)

    def forward(self, x):
        l1_out = relu(self.l1(x))
        l2_out = relu(self.l2(l1_out))
        l3_out = relu(self.l3(l2_out))
        return softmax(l3_out)

In [12]:
net = FeedForwardNet()
out = net(X_train)

In [13]:
def cross_entropy_loss(pred, target):
    # assert is_binary(target), "Target must contain all binary values"
    # assert is_stochastic(pred), "Array must sum to 1"
    
    return -slow.dot(target.T, slow.log(pred))

In [14]:
loss = cross_entropy_loss(out, y_train)

In [15]:
loss.mean().backward()

AssertionError: Tensor does not require grad